<a href="https://colab.research.google.com/github/4dsolutions/clarusway_data_analysis/blob/main/python_warm_up/warmup_object_sql.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a><br/>
[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/4dsolutions/clarusway_data_analysis/blob/main/python_warm_up/warmup_object_sql.ipynb)

# SQLite3 and Context Managers

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/52563704012/in/album-72177720296706479/" title="LMS Dashboard"><img src="https://live.staticflickr.com/65535/52563704012_71ef4beb8a_b.jpg" width="1024" height="354" alt="LMS Dashboard"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

Python Warm-up Notebooks:

*  [Introduction to Python](warmup_python_intro.ipynb)
*  [3rd Party Libraries](warmup_3rd_party_datascience.ipynb)
*  [Object Types](warmup_data_structures.ipynb)
*  [Object Oriented Paradigm](warmup_object_oriented.ipynb)
*  [Calling Callables and Type Checking](warmup_callables.ipynb)
*  [Class and Static Methods, Properties](warmup_object_oriented2.ipynb)
*  [SQLite3 and Context Managers](warmup_object_sql.ipynb) (you are here)
*  [Iterators and Generators](warmup_generators.ipynb)

Database engines, such as MySQL, PostgreSQL, SQL Server and many more, expect to be talked to in a certain way.  They face the world through an API.  Python talks through this API via its DB API or database API.

What's an API?  Let's hear [from IBM](https://www.ibm.com/topics/api):

<blockquote>
An API, or application programming interface, is a set of defined rules that enable different applications to communicate with each other. It acts as an intermediary layer that processes data transfers between systems, letting companies open their application data and functionality to external third-party developers, business partners, and internal departments within their companies.
</blockquote>
    
Twitter, Facebook and other companies allow programmatic access to their content, which does not entail bypassing security.  Their APIs offer approved and secure ways of accessing data.  

However the term 'API' need to refer to companies offering data.  Any library or package tends to interface with the greater world through a documented set of valid requests.  SQL engines, i.e. databases the run SQL, are a case in point.

Included in Python's Standard Library is `sqlite3`, which is designed to work with [any SQLite database](https://www.sqlite.org/index.html), and which we give the alias `sql` in the code below.  

We have a certain `roller_coasters.db` file ready to connect to and select from.

In [1]:
import sqlite3 as sql

In [2]:
conn = sql.connect("roller_coasters.db")

In [3]:
cur = conn.cursor()  # the connection supplies a cursor type object

In [4]:
type(cur)

sqlite3.Cursor

A database usually consists of several tables, some user defined, others [containing meta data](https://www.sqlite.org/schematab.html) and defined by the SQL engine.  We are free to inspect.

In [5]:
result = cur.execute("SELECT name FROM sqlite_schema "
"WHERE type = 'table';")

table_names = result.fetchall()
table_names 

[('Coasters',)]

[Asking about the structure](https://www.sqlitetutorial.net/sqlite-describe-table/) of the table `Coasters` gets me the SQL I would need to recreate from scratch as an empty structure.  I would still need to insert all the records.

In [6]:
result = cur.execute("""SELECT sql 
FROM sqlite_schema 
WHERE name = 'Coasters';""")

In [7]:
table_structure = result.fetchall()

In [8]:
table_structure  # ugly list

[('CREATE TABLE Coasters\n            (Name text PRIMARY KEY, \n             Park text,\n             State text, \n             Country text,\n             Duration int,\n             Speed int,\n             Height int,\n             VertDrop int,\n             Length int,\n             Yr_Opened int,\n             Inversions int)',)]

In [9]:
print("".join(table_structure[0]))  # that's better

CREATE TABLE Coasters
            (Name text PRIMARY KEY, 
             Park text,
             State text, 
             Country text,
             Duration int,
             Speed int,
             Height int,
             VertDrop int,
             Length int,
             Yr_Opened int,
             Inversions int)


SQL supports creating, retrieving, updating, and deleting records (CRUD for short).  

Above we see what creating looks like.  

Here's selecting:

In [10]:
result = cur.execute("""SELECT Name, Park, State, Country 
FROM Coasters WHERE Country == 'Japan';""")

In [11]:
for rec in result.fetchall():
    print(rec)

('Titan', 'Space World', 'Kitakyushu', 'Japan')
('Steel Dragon 2000', 'Nagashima Spa Land', 'Nagashima', 'Japan')
('Fujiyama', 'Fuji-Q Highlands', 'FujiYoshida-shi', 'Japan')
('Thunder Dolphin', 'LaQua', 'Tokyo', 'Japan')
('Hayabusa', 'Tokyo SummerLand', 'Tokyo', 'Japan')


## Context Managers

Python comes with a `with` keyword.  The object that follows should be a context manager, meaning it contains `__enter__` and `__exit__` methods.

Here's a simple example:

In [12]:
class CM:
    
    def __init__(self):  # optional
        self.something = 10
        
    def __enter__(self):
        print("__enter__ triggered")
        return self.something  
    
    def __exit__(self, *oops):
        print("__exit__ triggered")

In [13]:
with CM() as target:
    print("What __enter__ returned: ", target)
    print("... still in the with block")
    
print("Outside the with block")

__enter__ triggered
What __enter__ returned:  10
... still in the with block
__exit__ triggered
Outside the with block


Lets use this design pattern to connect to a database by means of `with`.  Entering the context involves opening the database, and exiting the with block involves closing it.

In [14]:
class DB:
    
    def __init__(self, name):  # optional
        self.db = name
        
    def __enter__(self):
        print("opening db")
        self.conn = sql.connect(self.db)
        self.cur  = self.conn.cursor()
        return self  # ... as db  (db will name self)
    
    def __exit__(self, *oops):
        print("closing db")
        if oops == (None, None, None):  # no exceptions raised
            self.conn.close()
        else:
            print("Something went wrong")
            try:
                self.conn.close()  # will this work?
            except:
                raise              # if not, re-raise the exception

In [15]:
with DB("roller_coasters.db") as db:
    # db contains conn and cur
    result = db.cur.execute("SELECT Name, Park, State, Country "
                            "FROM Coasters WHERE Country == 'Japan';") # these strings concatenate
    for rec in result.fetchall():
        print(rec)

opening db
('Titan', 'Space World', 'Kitakyushu', 'Japan')
('Steel Dragon 2000', 'Nagashima Spa Land', 'Nagashima', 'Japan')
('Fujiyama', 'Fuji-Q Highlands', 'FujiYoshida-shi', 'Japan')
('Thunder Dolphin', 'LaQua', 'Tokyo', 'Japan')
('Hayabusa', 'Tokyo SummerLand', 'Tokyo', 'Japan')
closing db


A custom context manager, say a subclass of the above, might be hard-coded to work with a specific database, and be fleshed out with many more methods for using it, via the DB API.